In [1]:
import torch
import torch.nn as nn
import torchvision
from transformers import BertModel, BertTokenizer, AutoTokenizer, AdamW
from torch.utils.data import Dataset

import torchvision.transforms as transforms

from collections import Counter
from torch.utils.data import DataLoader
import functools
from sklearn.model_selection import train_test_split
from PIL import Image
import torch.optim as optim

import contextlib
import numpy as np
import random
import shutil
import os
import json
import pandas as pd
import logging
import time
from datetime import timedelta
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, recall_score, precision_score

model_id = "dumitrescustefan/bert-base-romanian-cased-v1"



c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ImageEncoder(nn.Module):
    def __init__(self, args):
        super(ImageEncoder, self).__init__()
        self.args = args
        model = torchvision.models.resnet152(weights='ResNet152_Weights.DEFAULT')
        modules = list(model.children())[:-2]
        self.model = nn.Sequential(*modules)

        pool_func = (nn.AdaptiveAvgPool2d)

        # if args.num_image_embeds in [1, 2, 3, 5, 7]:
        # self.pool = pool_func((args.num_image_embeds, 1))
        self.pool = pool_func((1, 1))

    def forward(self, x):
        # Bx3x224x224 -> Bx2048x7x7 -> Bx2048xN -> BxNx2048
        out = self.pool(self.model(x))
        out = torch.flatten(out, start_dim=2)
        out = out.transpose(1, 2).contiguous()
        return out  # BxNx2048


class ImageClf(nn.Module):
    def __init__(self, args):
        super(ImageClf, self).__init__()
        self.args = args
        self.img_encoder = ImageEncoder(args)
        self.clf = nn.Linear(2048 * 1, 5)

    def forward(self, x):
        x = self.img_encoder(x)
        x = torch.flatten(x, start_dim=1)
        out = self.clf(x)
        return out

In [3]:
class ImageBertEmbeddings(nn.Module):
    def __init__(self, args, embeddings, vocab):
        super(ImageBertEmbeddings, self).__init__()
        self.args = args
        self.vocab = vocab
        self.img_embeddings = nn.Linear(2048, 768)
        self.position_embeddings = embeddings.position_embeddings
        self.token_type_embeddings = embeddings.token_type_embeddings
        self.word_embeddings = embeddings.word_embeddings
        self.LayerNorm = embeddings.LayerNorm
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_imgs, token_type_ids):
        bsz = input_imgs.size(0)
        seq_length = 1 + 2  # +2 for CLS and SEP Token

        cls_id = torch.LongTensor([self.vocab.stoi["[CLS]"]]).cuda()
        cls_id = cls_id.unsqueeze(0).expand(bsz, 1)
        cls_token_embeds = self.word_embeddings(cls_id)

        sep_id = torch.LongTensor([self.vocab.stoi["[SEP]"]]).cuda()
        sep_id = sep_id.unsqueeze(0).expand(bsz, 1)
        sep_token_embeds = self.word_embeddings(sep_id)

        imgs_embeddings = self.img_embeddings(input_imgs)
        token_embeddings = torch.cat(
            [cls_token_embeds, imgs_embeddings, sep_token_embeds], dim=1
        )

        position_ids = torch.arange(seq_length, dtype=torch.long).cuda()
        position_ids = position_ids.unsqueeze(0).expand(bsz, seq_length)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        embeddings = token_embeddings + position_embeddings + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings


class MultimodalBertEncoder(nn.Module):
    def __init__(self, args, vocab):
        super(MultimodalBertEncoder, self).__init__()
        self.args = args
        self.vocab = vocab
        bert = BertModel.from_pretrained(model_id)
        self.txt_embeddings = bert.embeddings

        self.img_embeddings = ImageBertEmbeddings(args, self.txt_embeddings, vocab)
        self.img_encoder = ImageEncoder(args)
        self.encoder = bert.encoder
        self.pooler = bert.pooler
        self.clf = nn.Linear(768, 5)

    def forward(self, input_txt, attention_mask, segment, input_img):
        bsz = input_txt.size(0)
        attention_mask = torch.cat(
            [
                torch.ones(bsz, 1 + 2).long().cuda(),
                attention_mask,
            ],
            dim=1,
        )
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        extended_attention_mask = extended_attention_mask.to(
            dtype=next(self.parameters()).dtype
        )
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        img_tok = (
            torch.LongTensor(input_txt.size(0), 1 + 2)
            .fill_(0)
            .cuda()
        )
        img = self.img_encoder(input_img)  # BxNx3x224x224 -> BxNx2048
        img_embed_out = self.img_embeddings(img, img_tok)
        txt_embed_out = self.txt_embeddings(input_txt, segment)
        encoder_input = torch.cat([img_embed_out, txt_embed_out], 1)  # Bx(TEXT+IMG)xHID

        encoded_layers = self.encoder(
            encoder_input, extended_attention_mask
        )

        return self.pooler(encoded_layers[-1])


class MultimodalBertClf(nn.Module):
    def __init__(self, args, vocab):
        super(MultimodalBertClf, self).__init__()
        self.args = args
        self.vocab = vocab
        self.enc = MultimodalBertEncoder(args, self.vocab)
        self.clf = nn.Linear(768, 5)

    def forward(self, txt, mask, segment, img):
        x = self.enc(txt, mask, segment, img)
        return self.clf(x)

In [4]:
 # Definirea listelor pentru a stoca datele
folders = ['stiri_fabricate', 'stiri_plauzibile', 'stiri_propagandistice', 'stiri_reale', 'stiri_satirice']
image_folders = [folder + '_img' for folder in folders]
data = {'img_path': [], 'Text': [], 'Label': []}
 

image_dir = r'C:\Users\bebed\OneDrive\Desktop\Dizertatie\dataset\images'
root_dir = r'C:\Users\bebed\OneDrive\Desktop\Dizertatie\dataset'

classes = sorted(os.listdir(image_dir))
file_paths = []
labels = []
for i, class_name in enumerate(classes):
    class_path = os.path.join(image_dir, class_name)
    filenames = os.listdir(class_path)
    dir_name = "stiri_" + class_name
    text_dir = os.path.join(root_dir, dir_name)   
    for filename in filenames:
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(class_path, filename)
            data['img_path'].append(img_path)
            data['Label'].append(class_name)
            image_name_no_ext, _ = os.path.splitext(filename)
            text_file_path = os.path.join(text_dir, f'{image_name_no_ext}.txt')
            if os.path.exists(text_file_path):
                with open(text_file_path, 'r', encoding='utf-8') as file:
                    text_data = file.read()
                data['Text'].append(text_data)

# print(len(data["Text"]))


df = pd.DataFrame(data)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


# # Afișăm DataFrame-ul
# print(df)
 
train_json_filename = 'train_stiri_text_mapping_image.json'
test_json_filename = 'test_stiri_text_mapping_image.json'

train_df.to_json(train_json_filename, orient='records', lines=True, force_ascii=False)
test_df.to_json(test_json_filename, orient='records', lines=True, force_ascii=False)

# # Print a message indicating the successful save
print(f'DataFrame has been saved to {train_json_filename}')
print(f'DataFrame has been saved to {test_json_filename}')

json_filename = 'stiri_text_mapping_image.json'

# Load the JSON data from the file
data = []
with open(json_filename, 'r', encoding='utf-8') as file:
    for line in file:
        record = json.loads(line)
        data.append(record)


DataFrame has been saved to train_stiri_text_mapping_image.json
DataFrame has been saved to test_stiri_text_mapping_image.json


In [5]:
def get_labels_and_frequencies():
    class_count = {}
    for i, class_name in enumerate(classes):
        class_path = os.path.join(image_dir, class_name)
        filenames = os.listdir(class_path)
        class_count[class_name] = len(filenames)
    label_freqs = Counter(class_count)

    return list(label_freqs.keys()), label_freqs


class JsonlDataset(Dataset):
    def __init__(self, data_path, tokenizer, transforms, vocab, args):
        self.data = self.get_data(data_path)
        self.data_dir = os.path.dirname(data_path)
        self.tokenizer = tokenizer
        self.args = args
        self.classes = get_labels_and_frequencies()
        self.vocab = vocab
        self.n_classes = 5
        self.text_start_token = ["[SEP]"]

        self.max_seq_len = 512
        self.max_seq_len -= 1

        self.transforms = transforms

    def get_data(self, data_path):
        data = []
        with open(data_path, 'r', encoding='utf-8') as file:
            for line in file:
                record = json.loads(line)
                data.append(record)
        return data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = (
            self.text_start_token
            + self.tokenizer.tokenize(self.data[index]["Text"])[
                : (512 - 1)
            ]
        )
        segment = torch.zeros(len(sentence))

        sentence = torch.LongTensor(
            [
                self.vocab.stoi[w] if w in self.vocab.stoi else self.vocab.stoi["[UNK]"]
                for w in sentence
            ]
        )

        label = torch.LongTensor(
            [ self.classes[0].index(self.data[index]["Label"])]
        )

        image = None
        if self.data[index]["img_path"]:
            image = Image.open(self.data[index]["img_path"]).convert("RGB")
        image = self.transforms(image)

        # The first SEP is part of Image Token.
        segment = segment[1:]
        sentence = sentence[1:]
        # The first segment (0) is of images.
        segment += 1
        return sentence, segment, image, label

In [6]:
class Vocab(object):
    def __init__(self, emptyInit=False):
        if emptyInit:
            self.stoi, self.itos, self.vocab_sz = {}, [], 0
        else:
            self.stoi = {
                w: i
                for i, w in enumerate(["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
            }
            self.itos = [w for w in self.stoi]
            self.vocab_sz = len(self.itos)

    def add(self, words):
        cnt = len(self.itos)
        for w in words:
            if w in self.stoi:
                continue
            self.stoi[w] = cnt
            self.itos.append(w)
            cnt += 1
        self.vocab_sz = len(self.itos)

In [7]:
def get_transforms():
    return transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.46777044, 0.44531429, 0.40661017],
                std=[0.12221994, 0.12145835, 0.14380469],
            ),
        ]
    )



def get_vocab():
    vocab = Vocab()
    bert_tokenizer = BertTokenizer.from_pretrained(model_id)
    vocab.stoi = bert_tokenizer.vocab
    vocab.itos = bert_tokenizer.ids_to_tokens
    vocab.vocab_sz = len(vocab.itos)

    return vocab

def collate_fn(batch, args):
    lens = [len(row[0]) for row in batch]
    bsz, max_seq_len = len(batch), max(lens)

    mask_tensor = torch.zeros(bsz, max_seq_len).long()
    text_tensor = torch.zeros(bsz, max_seq_len).long()
    segment_tensor = torch.zeros(bsz, max_seq_len).long()

    img_tensor = None
    img_tensor = torch.stack([row[2] for row in batch])

    # Single Label case
    tgt_tensor = torch.cat([row[3] for row in batch]).long()

    for i_batch, (input_row, length) in enumerate(zip(batch, lens)):
        tokens, segment = input_row[:2]
        text_tensor[i_batch, :length] = tokens
        segment_tensor[i_batch, :length] = segment
        mask_tensor[i_batch, :length] = 1

    return text_tensor, segment_tensor, mask_tensor, img_tensor, tgt_tensor

def get_data_loaders(args):
    tokenizer = BertTokenizer.from_pretrained(model_id)
    transforms = get_transforms()

    # args.labels, args.label_freqs = get_labels_and_frequencies()
    vocab = get_vocab()
    # args.vocab = vocab
    # args.vocab_sz = vocab.vocab_sz
    # args.n_classes = len(args.labels)

    train_path = r'C:\Users\bebed\OneDrive\Desktop\BigData\project\train_stiri_text_mapping_image.json'
    test_path = r'C:\Users\bebed\OneDrive\Desktop\BigData\project\train_stiri_text_mapping_image.json'

    train = JsonlDataset(
        train_path,
        tokenizer,
        transforms,
        vocab,
        args,
    )

    # args.train_data_len = len(train)

    dev = JsonlDataset(
        test_path,
        tokenizer,
        transforms,
        vocab,
        args,
    )

    collate = functools.partial(collate_fn, args=args)

    train_loader = DataLoader(
        train,
        batch_size=4,
        shuffle=True,
        num_workers=0,
        collate_fn=collate,
    )

    val_loader = DataLoader(
        dev,
        batch_size=2,
        shuffle=False,
        num_workers=0,
        collate_fn=collate,
    )

    test_set = JsonlDataset(
        test_path,
        tokenizer,
        transforms,
        vocab,
        args,
    )

    test_loader = DataLoader(
        test_set,
        batch_size=4,
        shuffle=False,
        num_workers=0,
        collate_fn=collate,
    )


    return train, train_loader, dev, val_loader, test_set, test_loader



In [8]:
def get_criterion():
    criterion = nn.CrossEntropyLoss()
    return criterion

def get_optimizer(model):
    total_steps = (
        670
        / 32
        / 24
        * 100
    )
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], "weight_decay": 0.01},
        {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], "weight_decay": 0.0,},
    ]
    optimizer = AdamW(
        optimizer_grouped_parameters,
        lr=1e-4,
        no_deprecation_warning=True
    )
    return optimizer

def get_scheduler(optimizer):
    return optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, "max", patience=2, verbose=True, factor=0.5
    )

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [9]:
class LogFormatter:
    def __init__(self):
        self.start_time = time.time()

    def format(self, record):
        elapsed_seconds = round(record.created - self.start_time)

        prefix = "%s - %s - %s" % (
            record.levelname,
            time.strftime("%x %X"),
            timedelta(seconds=elapsed_seconds),
        )
        message = record.getMessage()
        message = message.replace("\n", "\n" + " " * (len(prefix) + 3))
        return "%s - %s" % (prefix, message)


def create_logger(filepath, args):
    # create log formatter
    log_formatter = LogFormatter()

    # create file handler and set level to debug
    file_handler = logging.FileHandler(filepath, "a")
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(log_formatter)

    # create console handler and set level to info
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(log_formatter)

    # create logger and set level to debug
    logger = logging.getLogger()
    logger.handlers = []
    logger.setLevel(logging.DEBUG)
    logger.propagate = False
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    # reset logger elapsed time
    def reset_time():
        log_formatter.start_time = time.time()

    logger.reset_time = reset_time

    logger.info(
        "\n".join(
            "%s: %s" % (k, str(v))
            for k, v in sorted(dict(vars(args)).items(), key=lambda x: x[0])
        )
    )

    return logger

In [15]:
def model_forward(i_epoch, model, criterion, batch):
    txt, segment, mask, img, tgt = batch

    freeze_img = i_epoch < 0
    freeze_txt = i_epoch < 0

    for param in model.enc.img_encoder.parameters():
        param.requires_grad = not freeze_img
    for param in model.enc.encoder.parameters():
        param.requires_grad = not freeze_txt

    txt, img = txt.cuda(), img.cuda()
    mask, segment = mask.cuda(), segment.cuda()
    out = model(txt, mask, segment, img)

    tgt = tgt.cuda()
    loss = criterion(out, tgt)
    return loss, out, tgt


def model_eval(i_epoch, data, model, criterion):
    with torch.no_grad():
        losses, preds, tgts = [], [], []
        for batch in data:
            loss, out, tgt = model_forward(i_epoch, model, criterion, batch)
            losses.append(loss.item())
            pred = torch.nn.functional.softmax(out, dim=1).argmax(dim=1).cpu().detach().numpy()

            preds.append(pred)
            tgt = tgt.cpu().detach().numpy()
            tgts.append(tgt)

    metrics = {"loss": np.mean(losses)}
    tgts = [l for sl in tgts for l in sl]
    preds = [l for sl in preds for l in sl]
    metrics["acc"] = accuracy_score(tgts, preds)
    metrics["macro_f1"] = f1_score(tgts, preds, average="macro")
    metrics["micro_f1"] = f1_score(tgts, preds, average="micro")
    metrics["recall"] = recall_score(tgts, preds, average='micro')
    metrics["precision"] = precision_score(tgts, preds, average = 'micro')
    # if store_preds:
    #     store_preds_to_disk(tgts, preds, args)

    return metrics

In [11]:

train, train_loader, dev, val_loader, test_set, test_loader = get_data_loaders([])

vocab = get_vocab()
model = MultimodalBertClf([], vocab)
model.cuda()


MultimodalBertClf(
  (enc): MultimodalBertEncoder(
    (txt_embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (img_embeddings): ImageBertEmbeddings(
      (img_embeddings): Linear(in_features=2048, out_features=768, bias=True)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (img_encoder): ImageEncoder(
      (model): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [12]:
criterion = get_criterion()
optimizer = get_optimizer(model)
scheduler = get_scheduler(optimizer)

set_seed(42)
start_epoch, global_step, n_no_improve, best_metric = 0, 0, 0, -np.inf

max_epochs = 10

In [17]:
for i_epoch in range(start_epoch, max_epochs):
    train_losses = []
    model.train()
    optimizer.zero_grad()
    for batch in tqdm(train_loader, total=len(train_loader)):
        loss, _, _ = model_forward(i_epoch, model, criterion, batch)
        loss = loss / 24

        train_losses.append(loss.item())
        loss.backward()
        global_step += 1
        if global_step % 24 == 0:
            optimizer.step()
            optimizer.zero_grad()

    model.eval()
    print("Train Loss: {:.4f}".format(np.mean(train_losses)))
    metrics = model_eval(i_epoch, train_loader, model, criterion)
    print("{}: Loss: {:.5f} | Acc: {:.5f} | macro_f1 {:.5f} | micro_f1 {:.5f} | recall {:.5f} | precision {:.5f}".format(
                "Train", metrics["loss"], metrics["acc"], metrics["macro_f1"], metrics["micro_f1"], metrics["recall"],  metrics["precision"]))
    
    tuning_metric = (metrics["acc"])
    scheduler.step(tuning_metric)
    is_improvement = tuning_metric > best_metric
    if is_improvement:
        best_metric = tuning_metric
        n_no_improve = 0
    else:
        n_no_improve += 1
    
    model.eval()
    test_metrics = model_eval(
        np.inf, test_loader, model, criterion
    )
    print("{}: Loss: {:.5f} | Acc: {:.5f} | macro_f1 {:.5f} | micro_f1 {:.5f} | recall {:.5f} | precision {:.5f}".format(
                "Test", test_metrics["loss"], test_metrics["acc"], test_metrics["macro_f1"], test_metrics["micro_f1"], test_metrics["recall"], test_metrics["precision"]))



 40%|███▉      | 67/168 [03:03<04:57,  2.95s/it]c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 55%|█████▌    | 93/168 [04:21<03:49,  3.07s/it]c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\PIL\TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 168/168 [07:37<00:00,  2.73s/it]


Train Loss: 0.0226
Train: Loss: 0.41271 | Acc: 0.83731 | macro_f1 0.70835 | micro_f1 0.83731 | recall 0.83731 | precision 0.83731


c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\PIL\TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))
c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Test: Loss: 0.41358 | Acc: 0.83731 | macro_f1 0.70835 | micro_f1 0.83731 | recall 0.83731 | precision 0.83731


 18%|█▊        | 30/168 [01:39<08:02,  3.50s/it]c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\PIL\TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))
 88%|████████▊ | 148/168 [08:12<01:09,  3.46s/it]c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 168/168 [09:19<00:00,  3.33s/it]


...

Test: Loss: 0.00618 | Acc: 0.99851 | macro_f1 0.99816 | micro_f1 0.99851 | recall 0.99851 | precision 0.99851


 56%|█████▌    | 94/168 [05:12<04:13,  3.42s/it]c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\PIL\TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))
 70%|███████   | 118/168 [06:51<03:54,  4.69s/it]c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 168/168 [09:42<00:00,  3.47s/it]


Train Loss: 0.0005
Train: Loss: 0.02351 | Acc: 0.99104 | macro_f1 0.98867 | micro_f1 0.99104 | recall 0.99104 | precision 0.99104


c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\PIL\TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))
c:\Users\bebed\anaconda3\envs\bigdataenv\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Test: Loss: 0.02350 | Acc: 0.99104 | macro_f1 0.98867 | micro_f1 0.99104 | recall 0.99104 | precision 0.99104


In [ ]:
# del model
# del optimizer
# torch.cuda.empty_cache()